### Install sentence transformers library

In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 71kB 5.4MB/s 
     |████████████████████████████████| 1.8MB 9.9MB/s 
     |████████████████████████████████| 1.2MB 31.5MB/s 
     |████████████████████████████████| 890kB 48.1MB/s 
     |████████████████████████████████| 2.9MB 51.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp36-none-any.whl size=103068 sha256=a37865dc327e71ad247767b601dce237c89023f053884af07c64bf67e6d90799
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=1a4285823f064643bf3cf85cdd1338285f477ca169955113bdcb1f658696ef92
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


### Useful imports

In [ ]:
import json,glob,nltk,copy,torch,time,sentence_transformers,pickle
import numpy as np
from scipy import spatial
from queue import PriorityQueue
from sentence_transformers import SentenceTransformer,util
nltk.download('punkt')
from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Mounted at /content/drive


### Retrieve dataset

In [ ]:
!wget https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2020-03-13.tar.gz
!tar -xf cord-19_2020-03-13.tar.gz
!tar -xf 2020-03-13/comm_use_subset.tar.gz

--2021-02-03 15:16:42--  https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2020-03-13.tar.gz
Resolving ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)... 52.218.234.233
Connecting to ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)|52.218.234.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278921140 (266M) [application/x-tar]
Saving to: ‘cord-19_2020-03-13.tar.gz’

cord-19_2020-03-13. 100%[===================>] 266.00M  61.7MB/s    in 4.6s    

2021-02-03 15:16:47 (57.9 MB/s) - ‘cord-19_2020-03-13.tar.gz’ saved [278921140/278921140]



### Prepare GPU Cuda.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device available for running: ")
print(device)

Device available for running: 
cuda


### Read JSON files and store title,abstract and text of each article into a list

In [ ]:
data = []

files = glob.glob('comm_use_subset/*', recursive=True)
number_of_articles = len(files)
bound = 9000

for single_file in files[0:bound]:
  with open(single_file, 'r') as f:
    json_file = json.load(f)
    
    # Retrieve title
    title = json_file["metadata"]["title"]

    # Retrieve abstracts
    abstracts = []
    if len(json_file["abstract"]) != 0 :
      for abstract in json_file["abstract"]:
        abstracts.append(abstract["text"])

    # Retrieve texts
    texts = []
    for text in json_file["body_text"]:
      texts.append(text["text"])

    data.append([title,abstracts,texts])

### Convert corpus to sentences with help of library nltk

In [ ]:
# For each article
for article in range(bound):
 
  # Abstracts section 
  for abstract in range(len(data[article][1])): 
    data[article][1][abstract] = nltk.sent_tokenize(data[article][1][abstract])
   
  # Texts section
  for text in range(len(data[article][2])):
    data[article][2][text] = nltk.sent_tokenize(data[article][2][text])

### Transform sentences to embeddings with first sentence trasformer 'stsb-bert-base' model

In [ ]:
# We gonna calculate time spent for sentence embeddings
fist_start_time = time.time()

# Declare our first sentence transformer model and pass it to appropriate device
first_model = SentenceTransformer('stsb-bert-base').to(device)

# Here we'll put sentence embeddings of each article from first sentence transformer model
first_sentence_embeddings = [[] for i in range(bound)]

# Here we'll put for each article sentences of abstract and body text
flatten_data = [[] for i in range(bound)]

# For each article
for article in range(bound):
  
  # Get abstract and body text of each article
  abstract_ = data[article][1]
  body_ = data[article][2]

  # Process to keep abstract and body text's sentences of each article in a big list
  for abstract in abstract_:
    for sentence in abstract:
      flatten_data[article].append(sentence)
  for text in body_:
    for sentence in text:
      flatten_data[article].append(sentence)

  # Transform sentences to embeddings 
  first_sentence_embeddings[article].append(first_model.encode(flatten_data[article],convert_to_tensor=True))

  # Convert all sentence embeddings to a 2D pytorch tensor
  first_sentence_embeddings[article] = torch.cat(first_sentence_embeddings[article]) 
 
# Check elapsed time of first model
first_model_time = (time.time() - fist_start_time)/60
print("Elapsed time: %s minutes" % (round(first_model_time,1)))

# # Save first sentence embeddings
# with open('/content/drive/MyDrive/first_sentence_embeddings.pkl', 'wb') as f:
#   pickle.dump(first_sentence_embeddings,f)

Elapsed time: 110.3 minutes


### Random printings just for safety reasons

In [ ]:
# Testing with prints
print(len(first_sentence_embeddings),type(first_sentence_embeddings))
print(len(first_sentence_embeddings[0]),type(first_sentence_embeddings[0]))
print(first_sentence_embeddings[0].shape)

print(len(flatten_data),type(flatten_data))
print(len(flatten_data[0]),type(flatten_data[0]))
print(len(flatten_data[0][0]),type(flatten_data[0][0]))
print(flatten_data[0][0])

9000 <class 'list'>
140 <class 'torch.Tensor'>
torch.Size([140, 768])
9000 <class 'list'>
140 <class 'list'>
150 <class 'str'>
The essential requirement of the lymphotoxin beta receptor (LTβR) in the development and maintenance of peripheral lymphoid organs is well recognized.


### Declare our queries and tranform them to embeddings based on our two models.

In [ ]:
queries = ['What are the coronoviruses?','What was discovered in Wuhuan in December 2019?',
           'What is Coronovirus Disease 2019?','What is COVID-19?','What is caused by SARS-COV2?',
           'How is COVID-19 spread?','Where was COVID-19 discovered?','How does coronavirus spread?']

first_queries_embeddings = first_model.encode(queries,convert_to_tensor=True)

print("For 1st model... Number Of Queries:",len(first_queries_embeddings)," Query Embedding's Length:",len(first_queries_embeddings[0]))

For 1st model... Number Of Queries: 8  Query Embedding's Length: 768


### Test our first model

In [ ]:
# # Load first sentence embeddings
# with open('/content/drive/MyDrive/first_sentence_embeddings.pkl', 'rb') as f:
#   first_sentence_embeddings = pickle.load(f)

# We gonna calculate time spent for finding best answer
fist_start_time = time.time()

# For each query 
for qindex,query in enumerate(first_queries_embeddings):
  # Declare an empty priority queue
  answer_pq = PriorityQueue()
  # For each article's sentence embedding
  for index,embed in enumerate(first_sentence_embeddings):
    # Find the most similar vector and return it so as to add it to priority queue
    first_results = sentence_transformers.util.semantic_search(query,embed,top_k=1)
    for res in first_results:
      # Add to priority queue triple value of (score,article's index,sentence's index)
      answer_pq.put((-res[0]['score'],index,res[0]['corpus_id']))
  # Get vectors with the best cosine similarity (which are our answers)
  print("Query:",queries[qindex],"\n")
  for idx in range(4):  
    res = answer_pq.get()
    print("Answer",idx+1,":",flatten_data[res[1]][res[2]],"\nFrom article:",data[res[1]][0],"\nScore:",-res[0],"\n")
  print("------------------------------------------------------------------------------------------------------------------------------")

first_model_time = (time.time() - fist_start_time)/60
print("Elapsed time: %s minutes" % (round(first_model_time,1)))

Query: What are the coronoviruses? 

Answer 1 : in orthomyxoviruses (e.g. 
From article: Mapping overlapping functional elements embedded within the protein-coding regions of RNA viruses 
Score: 0.7579688 

Answer 2 : C) Parechoviruses. 
From article: A viral metagenomic survey identifies known and novel mammalian viruses in bats from Saudi Arabia 
Score: 0.7296622 

Answer 3 : reoviruses and orthomyxoviruses). 
From article: Non-canonical translation in RNA viruses 
Score: 0.6961699 

Answer 4 : Adenoviruses. 
From article: Virus-induced exacerbations in asthma and COPD 
Score: 0.6928731 

------------------------------------------------------------------------------------------------------------------------------
Query: What was discovered in Wuhuan in December 2019? 

Answer 1 : In December 2019 
From article: Clinical Medicine Characteristics of and Public Health Responses to the Coronavirus Disease 2019 Outbreak in China 
Score: 0.7151003 

Answer 2 : (2019c) , Zhao et al. 
From a

### Notes & Conclusions

>Note that our dataset is the initial-first release of CORD-19 dataset, 2020-03-13, which is the smallest dataset with 9000 articles. 
You can find it here: [CORD-19_Releases](https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases.html)

#### Just a recap of what we presented in this notebook!


>If we could briefly refer this notebook's approach we would say that we followed a brute force approach.So let us be more detailed...   
1. Initially, we discover what does each json file includes and we decided to retrieve as much as possible information for each 
article. So we obtained title,abstract and body text section of each article and store them to list.
2. Right after we did the process of corpus's tokenization to sentences. We continued by transforming all sentences of each abstract 
and body text section of each article to sentence embeddings. Here the obvious question arises... Why we didn't transform title 
section? We did experiment with title section, but we observed that at most of times there isn't so much information which pairs or
matches with our questions. Note that all embeddings have been converted to pytorch tensors!
3. After transforming queries to embeddings, we compare each sentence's embedding of each abstract's and body text's section of each 
article with each every query embedding, via function sentence_transformers.util.semantic_search, which innerly utilizes cosine 
similarity. -Basically, this function retrieves from us many sentences embeddings and a single query embedding and returns as topk 
best (closest by cosine similarity) sentence embeddings-. So we can understand that we search everything in order to find the most 
appropriate answer!
4. Here we follow a more brute force way, in order to search every possible information from articles, but that 
cost us to time, approximately whole process endured 2 hours, with total use of Cuda GPU. In addition, just to remind in 
this notebook we used pretrained sbert model,'stsb-bert-base', in order to transform sentences to embeddings of size 768. 



#### Notes about our results!

>Our first thought before we even begin was that with our brute force approach, we may be more time and computer power consuming, but 
we are going to retrieve sentences with the best cosine similarities that we could have found. But, something that we came up was 
that cosine similarity is a metric which compares angle of two vectors and returns a number between 0 and 1 which indicates how much 
similar those two vectors are. This leads us to the result that in our problem, in order to be more similar two embeddings, they 
must have as much as possible same words (between a query and an answer). So, we can conclude that this will have a glance of 
textual similarity rather than question-answering. And from theory's belief we can prove that practically our results!

>For example, in query: What are the coronoviruses?

>We gain answers like: 
>1. "in orthomyxoviruses (e.g."
>2. "C) Parechoviruses."
>3. "reoviruses and orthomyxoviruses)."  
>4. "Adenoviruses."

>We can obviously notice that all 4 answers, which gave cosine similarity approximately 0.69-0.75, have as a part of their answer 
the subword "viruses". And that happened because it indeed exists in our query, in word "corono"-"viruses". From that simple 
example, we can figure out that instead of an answer like "Coronoviruses are ...", we obtained as answer something that it is like 
or similar some words of query and in addition to that answers contain few words. This phenomenon happens at most times, because 
the less words a sentence have the less compares are gonna take place with word "viruses". So let's take for example two simple 
answers:

>>a) "Adenoviruses."

>>b) "Coronaviruses are a large family of viruses that are known to cause illness ranging from the common cold."

>Note that second answer is the first answer we found in Google, so it's a gold answer! But unfortunately if we compare cosine 
similarity scores between our query and the above answers, first answer will 'win'. This can be explained by the fact that sentence 
embedding of second answer isn't so close to our query,for the reason that sentence embedding of "Adenoviruses" is closer in vector
space than embedding of sentence with too 'confused' or seemingly 'unrelated' words. 

>Unfortunately, at most of our questions we can observe the exact same behavior! Another example, which can prove that is:

>>Query: How does coronavirus spread?

>>Answers: 
>>1. "Coronaviruses: what are they?"
>>2. "Coronavirus."
>>3. "coronavirus."
>>4. "Coronavirus."

>>With cosine similarity scores between 0.75-0.82.

>But by observing thoroughly our questions-answers we found some exceptions, like this example:

>>Query: What is Coronovirus Disease 2019? 

>>Answers: 
>>1. "The spectrum of this disease in humans, now named coronavirus disease 2019 (COVID-19) [5] , is yet to be fully determined."
>>4. "The disease caused by SARS-CoV-2 was called "coronavirus disease 2019" (COVID-19) [7] ."
With cosine similarity scores between 0.60-0.62. 

>Compared with most percentage of answers, here we have more allowable answers. Precisely fourth answer is absolutely acceptable and 
relevant, altough it has the lowest cosine similarity among the four best answers!